# Imports

In [17]:
import warnings
warnings.filterwarnings("ignore")

# Load variables
import os
from dotenv import load_dotenv
load_dotenv()

# Snowpark Imports
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F

# Connect to Snowflake

In [18]:
snowflake_connection_cfg = {
    "ACCOUNT": os.getenv('SF_ACCOUNT'),
    "USER": os.getenv('SF_USER'),
    "ROLE": os.getenv('SF_ROLE'),
    "PASSWORD": os.getenv('SF_PASSWORD'),
    "DATABASE": os.getenv('SF_DATABASE'),
    "SCHEMA": os.getenv('SF_SCHEMA'),
    "WAREHOUSE": os.getenv('SF_WAREHOUSE')
}

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

print('Role:     ', session.get_current_role())
print('Warehouse:', session.get_current_warehouse())
print('Database: ', session.get_current_database())
print('Schema:   ', session.get_current_schema())

Role:      "ACCOUNTADMIN"
Warehouse: "COMPUTE_WH"
Database:  "MACHINE_LEARNING"
Schema:    "PUBLIC"


In [19]:
# Create some test data to work with
long_texts = [
    """
    The History of Internet Development: The internet, as we know it today, began as a military research project in the late 1960s. 
    Initially designed to share information securely among military personnel, it evolved into a tool for scientists to exchange data. 
    Over decades, what started as a rudimentary network of computers has transformed into a global system that connects billions of devices. 
    This massive growth was largely due to the advent of the World Wide Web in the early 1990s, which made the internet more accessible and user-friendly. 
    Websites proliferated rapidly, and the internet became a central medium for communication, entertainment, and commerce. 
    Today, internet technology continues to evolve with trends like the Internet of Things (IoT) and the increasing demand for faster, more reliable connections.
    """,
    """
    Advances in Mobile Technology: Mobile technology has seen unprecedented growth over the last two decades. 
    The transformation from basic cell phones to sophisticated smartphones is one of the most significant innovations in modern technology. 
    Early mobile phones were primarily used for voice communication and short text messages. However, the introduction of smartphones changed the landscape completely. 
    Modern smartphones offer a wide range of functions including internet access, high-quality photography, and mobile applications that cater to every imaginable need, from navigating one's way in a new city to managing personal finances or even monitoring health and fitness. 
    As mobile technology continues to advance, we see the integration of AI and augmented reality, pushing the boundaries of what smartphones can do.
    """,
    """
    The Role of Technology in Modern Education: Technology has dramatically transformed the educational landscape. 
    From elementary classrooms to prestigious universities, technology is an integral part of how education is delivered and received today. 
    Digital tools such as tablets, smartboards, and educational software have become commonplace, making learning more interactive and accessible. 
    Moreover, the rise of online education platforms has made it possible for people around the world to learn anything from coding to art history from the comfort of their homes. 
    These platforms offer courses from top institutions and industry leaders, making quality education more accessible than ever before. 
    While technology in education presents numerous opportunities, it also brings challenges such as digital divide and concerns about screen time.
    """
]


df = session.create_dataframe(long_texts, schema=['DOCUMENTS'])
df.show(n=15, max_width=1000)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DOCUMENTS"                                                                                                                                                                                                                                                                             |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|                                                                                                                                                      

# Register & Run Summarization Model

In [20]:
# Get the model registry object
from snowflake.ml.registry import Registry
reg = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema()
    )

In [21]:
# Get the summarization model from Huggingface
# Make sure it fits into a Snowflake warehouse and does not require GPUs
# Otherwise the model must deployed in Snowpark Container Services
from transformers import pipeline
pipe = pipeline("summarization", model="Falconsai/text_summarization")

# Register the model to Snowflake (predict is the model's function we want to call)
snow_model_custom = reg.log_model(
    pipe, 
    model_name='text_summarization', 
    sample_input_data=df.limit(10),
    conda_dependencies=['tokenizers','transformers','sentencepiece']
    )

# Outputs
summaries = snow_model_custom.run(df).cache_result()
summaries.select('SUMMARY_TEXT').show(max_width=1000)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'min_length': 30, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY_TEXT"                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
----------------------------------------------------------------------------------------------